In [ ]:
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = "https://www.i3s.unice.fr/~riveill/dataset/petfinder-adoption-prediction/"

In [ ]:
breeds = pd.read_csv(path+'breed_labels.csv')
colors = pd.read_csv(path+'color_labels.csv')
states = pd.read_csv(path+'state_labels.csv')

train = pd.read_csv(path+'train.csv')

train['dataset_type'] = 'train'

In [ ]:
len(train)

In [ ]:
# In this example notebook, we will only work with a small part of the dataset
N = 500
train = train[:N]

# Data Preprocessing

In [ ]:
if 'dataset_type' in train.columns:
    train = train.drop(labels='dataset_type', axis=1)
train.columns

In [ ]:
y_train = train['target']
X_train = train.drop(['target'], axis=1)
X_train.head()

In [ ]:
y_train.head()

In [ ]:
cat_cols = ['Type', 'Gender', 'Breed', 'Color1', 'Color2', 'Color3', 
       'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health']
num_cols = ['Age', 'Fee']
txt_cols = ['Description']
img_cols = ['Images']

In [ ]:
# Build the image list of the training set 
img_dir = "train_images/"
X_train['Images'] = [path+img_dir+img for img in train['Images']]

In [ ]:
from skimage import io

# Read the first image of the list
img = io.imread(X_train['Images'][0])
# have a look to the image
plt.imshow(img)

In [ ]:
!pip install opencv-python==4.5.5.62 opencv-contrib-python==4.5.5.62

In [ ]:
# convert the image to grey levels 
import cv2

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
# compute SIFT detector and descriptors
sift = cv2.SIFT_create()
kp,des = sift.detectAndCompute(gray,None)

In [ ]:
# plot image and descriptors
cv2.drawKeypoints(img,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(img)

In [ ]:
# First step, extract the SIFTs of each image
# Be carefull: very long process

def extract_SIFT(img_lst):
    nbSIFTs = 0    # Nomber of SIFTs
    SIFTs = []  # List of SIFTs descriptors 
    #dimImgs = []   # Nb of descriptors associated to each images

    for pathImg in tqdm(img_lst, position=0, leave=True): 
        img = io.imread(pathImg)
        if len(img.shape)==2: # this is a grey level image
            gray = img
        else: # we expect the image to be a RGB image or RGBA
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        kp, des = sift.detectAndCompute(gray, None)
        if len(kp) == 0 and img.shape[2]==4: #some images are mask on alpha channel: we thus extract this channel if not kpts have been detected
            gray = img[:,:,3]
            sift = cv2.SIFT_create()
            kp, des = sift.detectAndCompute(gray, None)
        
        nbSIFTs += des.shape[0]
        SIFTs.append(des)
        #dimImgs.append(des.shape[0])
    return nbSIFTs, SIFTs#, dimImgs

In [ ]:
nbSIFTs, SIFTs = extract_SIFT(X_train['Images'])
print('nbSifts: ', nbSIFTs)

In [ ]:
# Step 2: clusterize the SIFT
from sklearn.cluster import MiniBatchKMeans

def clusterize(SIFTs, nb_img_features=5, verbose=False):
    clusterizer = MiniBatchKMeans(n_clusters=nb_img_features)   # nb_img_features is a hyperparameter
    # learning of the clustering
    flat_list = SIFTs[0]
    for des in SIFTs[1:]:
        flat_list = np.concatenate((flat_list, des))
        if verbose:
            print("shape:", des.shape, flat_list.shape)
    clusterizer.fit(flat_list)
    # we now know the label of each SIFT descriptor
    return clusterizer

In [ ]:
clusterizer = clusterize(SIFTs, verbose=True)

In [ ]:
# Step 3: build the BOW representation of each images (i.e. construction of the BOFs)

def build_BOFs(SIFTs, clusterizer, verbose=False):
    ok, nok = 0, 0
    #BOF initialization
    nb_img_features = clusterizer.get_params()['n_clusters']
    BOFs = np.empty(shape=(0, nb_img_features), dtype=int)

    # Build label list
    flat_list = SIFTs[0]
    for des in SIFTs[1:]:
        flat_list = np.concatenate((flat_list, des))
        if verbose:
            print("shape:", des.shape, flat_list.shape)
    labels = clusterizer.predict(flat_list)

    # loop on images
    i = 0 # index for the loop on SIFTs
    for des in SIFTs:
        #initialisation of the bof for the current image
        tmpBof = np.array([0]*nb_img_features)
        j = 0
        # for every SIFT of the current image:
        nbs = des.shape[0]
        while j < nbs:
            tmpBof[labels[i]] += 1
            j+=1
            i+=1
        BOFs = np.concatenate((BOFs, tmpBof.reshape(1,-1)), axis=0)
    if verbose:
        print("BOFs : ", BOFs)
    
    return BOFs

In [ ]:
BOFs = build_BOFs(SIFTs, clusterizer, verbose=True)
BOFs.shape

In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin

class BOF_extractor(BaseEstimator,TransformerMixin):    
    def __init__(self, nb_img_features=10, verbose=False):
        self.nb_img_features = nb_img_features
        self.verbose = verbose
        self.path = path
        if self.verbose:
            print("BOF.init()")
        
    def fit(self, X, y=None):
        if self.verbose:
            print("BOF.fit()")
        nbSIFTs, SIFTs = extract_SIFT(X)
        self.clusterizer = clusterize(SIFTs, self.nb_img_features, self.verbose)
        
    def transform(self, X, y=None):
        if self.verbose:
            print("BOF.transform()")
        nbSIFTs, SIFTs = extract_SIFT(X)
        if self.verbose:
            print("nbSIFTs:", nbSIFTs)
        return build_BOFs(SIFTs, self.clusterizer, self.verbose)
    
    def fit_transform(self, X, y=None):
        if self.verbose:
            print("BOF.fit_transform()")
        nbSIFTs, SIFTs = extract_SIFT(X)
        self.clusterizer = clusterize(SIFTs, self.nb_img_features, self.verbose)
        return build_BOFs(SIFTs, self.clusterizer, self.verbose)

In [ ]:
test_BOF_extractor = BOF_extractor(nb_img_features=5, verbose=True)

In [ ]:
test_BOF_extractor.fit(X_train['Images'])

In [ ]:
BOFs = test_BOF_extractor.transform(X_train['Images'])
BOFs.shape

In [ ]:
BOFs = test_BOF_extractor.fit_transform(X_train['Images'])
BOFs.shape

In [ ]:
test = pd.read_csv(path+"test.csv")
y_test = test['target']
X_test = test.drop(['target'], axis=1)

img_dir = "test_images/"
X_test['Images'] = [path+img_dir+img for img in test['Images']]
len(X_test)

In [ ]:
BOFs = test_BOF_extractor.transform(X_test['Images'])
BOFs.shape

# Use GridSearchCV for each Classifier to get best model

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
text_preprocessor = CountVectorizer()
image_preprocessor = BOF_extractor(nb_img_features=3, verbose=False)

preprocessor = ColumnTransformer([
    ("categorical encoding", categorical_preprocessor, cat_cols),
    ("numerical encoding", numerical_preprocessor, num_cols),
    ("text encoding", text_preprocessor, 'Description'),
    ("image encoding", image_preprocessor, 'Images'),
])

classifier = LogisticRegression()

model = make_pipeline(preprocessor, classifier)

In [ ]:
test = pd.read_csv(path+"test.csv")

y_test = test['target']
X_test = test.drop(['target'], axis=1)

img_dir = "test_images/"
X_test['Images'] = [path+img_dir+img for img in test['Images']]
print("Test size:", len(X_test))

In [ ]:
#Support vector machine classifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier",SVC())
])
# make lists of different parameters to check
parameters = {
              'classifier__C': [ 1000,0.1, 10, 100, 1],
              'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'classifier__kernel': ['rbf','linear']
  }
grid_search_svm = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_svm.fit(X_train, y_train)
print("Best score:")
print(grid_search_svm.best_score_)
print("Best parameters found:")
print(grid_search_svm.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:54<00:00,  4.62it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:47<00:00,  4.64it/s]


Best score:
0.612
Best parameters found:
{'classifier__C': 0.1, 'classifier__gamma': 0.01, 'classifier__kernel': 'linear'}


In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier",LogisticRegression())
])
parameters = {
              'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear'],
              'classifier__penalty': ['l2'],
              'classifier__C': [100, 10, 1.0, 0.1, 0.01]
  }
grid_search_lr = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_lr.fit(X_train, y_train)
print("Best score:")
print(grid_search_lr.best_score_)
print("Best parameters found:")
print(grid_search_lr.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:49<00:00,  4.55it/s]


Best score:
0.6
Best parameters found:
{'classifier__C': 0.1, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}


In [ ]:
#Ridge Classifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeClassifier
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier",RidgeClassifier())
])

parameters = {
              'classifier__alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  }
grid_search_rc = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_rc.fit(X_train, y_train)
print("Best score:")
print(grid_search_rc.best_score_)
print("Best parameters found:")
print(grid_search_rc.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:51<00:00,  4.47it/s]


Best score:
0.5780000000000001
Best parameters found:
{'classifier__alpha': 0.7}


In [ ]:
#Knn
from sklearn.neighbors import KNeighborsClassifier
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier",KNeighborsClassifier())
])
parameters = {
              'classifier__n_neighbors': range(1, 21, 2),
              'classifier__weights': ['uniform', 'distance'],
              'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
  }

grid_search_knn = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_knn.fit(X_train, y_train)
print("Best score:")
print(grid_search_knn.best_score_)
print("Best parameters found:")
print(grid_search_knn.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [00:52<00:00,  4.73it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:47<00:00,  4.65it/s]


Best score:
0.558
Best parameters found:
{'classifier__metric': 'euclidean', 'classifier__n_neighbors': 7, 'classifier__weights': 'distance'}


In [ ]:
#Bagging decision trees
from sklearn.ensemble import BaggingClassifier
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier",BaggingClassifier())
])
parameters = {
              'classifier__n_estimators': [10, 100, 1000]
  }


grid_search_bdt = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_bdt.fit(X_train, y_train)
print("Best score:")
print(grid_search_bdt.best_score_)
print("Best parameters found:")
print(grid_search_bdt.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:50<00:00,  4.51it/s]


Best score:
0.626
Best parameters found:
{'classifier__n_estimators': 1000}


In [ ]:
#Random forest classifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier", RandomForestClassifier())
])
parameters = {
              'classifier__n_estimators': [10, 100, 1000],
              'classifier__max_features': ['sqrt', 'log2']
  }

grid_search_rfc = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_rfc.fit(X_train, y_train)
print("Best score:")
print(grid_search_rfc.best_score_)
print("Best parameters found:")
print(grid_search_rfc.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [02:06<00:00,  3.97it/s]


Best score:
0.634
Best parameters found:
{'classifier__max_features': 'sqrt', 'classifier__n_estimators': 1000}


In [ ]:
#Stochastic Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("classifier", GradientBoostingClassifier())
])
parameters = {
              'classifier__n_estimators': [10, 100, 1000],
              'classifier__learning_rate': [0.001, 0.01, 0.1],
              'classifier__subsample': [0.5, 0.7, 1.0],
              'classifier__max_depth': [3, 7, 9]
  }

grid_search_stb = GridSearchCV(pipe, param_grid=parameters, cv=2)
grid_search_stb.fit(X_train, y_train)
print("Best score:")
print(grid_search_stb.best_score_)
print("Best parameters found:")
print(grid_search_stb.best_params_)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:03<00:00,  3.94it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:02<00:00,  4.02it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [01:46<00:00,  4.70it/s]


Best score:
0.644
Best parameters found:
{'classifier__learning_rate': 0.01, 'classifier__max_depth': 7, 'classifier__n_estimators': 100, 'classifier__subsample': 0.7}


# Evaluation of the models

In [ ]:
from sklearn.svm import SVC
svc = SVC(C = 0.1, gamma = 0.01, kernel = 'linear')
model_svc = make_pipeline(preprocessor, svc)
model_svc.fit(X_train,y_train)
print('Score on training set: ')
print(model_svc.score(X_train, y_train))
print('Score on test set: ')
print(model_svc.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [02:06<00:00,  3.95it/s]


0.966
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:06<00:00,  3.75it/s]


0.596


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C = 0.1, penalty = 'l2', solver = 'liblinear')
model_lr = make_pipeline(preprocessor, lr)
model_lr.fit(X_train,y_train)
print('Score on training set: ')
print(model_lr.score(X_train, y_train))
print('Score on test set: ')
print(model_lr.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

0.936
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:02<00:00,  3.97it/s]


0.592


In [ ]:
from sklearn.linear_model import RidgeClassifier
rc = RidgeClassifier(alpha = 0.7)
model_rc = make_pipeline(preprocessor, rc)
model_rc.fit(X_train,y_train)
print('Score on training set: ')
print(model_rc.score(X_train, y_train))
print('Score on test set: ')
print(model_rc.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

0.998
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:01<00:00,  4.04it/s]


0.572


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(metric='euclidean', n_neighbors = 7, weights = 'distance')
model_knn = make_pipeline(preprocessor, knn)
model_knn.fit(X_train,y_train)
print('Score on training set: ')
print(model_knn.score(X_train, y_train))
print('Score on test set: ')
print(model_knn.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [02:06<00:00,  3.94it/s]


1.0
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:03<00:00,  3.95it/s]


0.528


In [ ]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(n_estimators = 1000)
model_bc = make_pipeline(preprocessor, bc)
model_bc.fit(X_train,y_train)
print('Score on training set: ')
print(model_bc.score(X_train, y_train))
print('Score on test set: ')
print(model_bc.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

1.0
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:02<00:00,  4.01it/s]


0.612


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features = 'sqrt', n_estimators = 1000)
model_rf = make_pipeline(preprocessor, rf)
model_rf.fit(X_train,y_train)
print('Score on training set: ')
print(model_rf.score(X_train, y_train))
print('Score on test set: ')
print(model_rf.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

1.0
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:02<00:00,  3.99it/s]


0.616


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 7, n_estimators = 100, subsample = 0.7)
model_gb = make_pipeline(preprocessor, gb)
model_gb.fit(X_train,y_train)
print('Score on training set: ')
print(model_gb.score(X_train, y_train))
print('Score on test set: ')
print(model_gb.score(X_test, y_test))

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

Score on training set: 


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

0.94
Score on test set: 


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:01<00:00,  4.03it/s]


0.596


In [ ]:
from joblib import dump, load

dump(model_rf, 'saoud.joblib')

['saoud.joblib']